# Visualize suspected leukemia cells

#### This notebook labels each cell within this patient's images by it's Phenograph cluster number generated in Phenograph-CD45_features.ipynb. Suspected Lk cells are outlined in red to visually confirm.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import cv2
from PIL import Image
from PIL import ImageDraw
from skimage import data, color, io, img_as_float
from os import listdir
from os.path import isfile, join
import skimage.io
import skimage.util
from skimage import data, segmentation, util, measure
from skimage.segmentation import expand_labels

In [2]:
# create dictionary of directories for each patient
img_dirs = {'P51':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P51_R1158_S1_Normalized_DAPI_B2M_CD34/', 
            'P52':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P52_R1149_S1_Normalized_DAPI_B2M_CD34/', 
            'P53':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P53_R1149_S2_Normalized_DAPI_B2M_CD34/', 
            'P56':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized_DAPI_B2M_CD34/', 
            'P57':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P57_R1158_S3_Normalized_DAPI_B2M_CD34/', 
            'P58':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized_DAPI_B2M_CD34/'}

In [3]:
patient = 'P53'

In [4]:
# create dictionary of directories for each patient
mask_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/2_Segmentation/3_NucMemMerging/' + patient + '_hybrid/labels_predicted_2_15_24/'

In [5]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + '_CD34_subclusters.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters
0,P53_FOV01_cell_3,1,3,A,1951.0,932.666667,1.0,0,16
1,P53_FOV01_cell_10,1,10,A,1552.0,1307.444444,1.0,0,0
2,P53_FOV01_cell_14,1,14,A,1636.0,1000.277778,1.0,0,7
3,P53_FOV01_cell_44,1,44,A,2457.0,963.777778,1.0,0,11
4,P53_FOV01_cell_63,1,63,A,2309.0,1052.222222,1.0,0,11
...,...,...,...,...,...,...,...,...,...
9917,P53_FOV25_cell_7810,25,7810,C,1130.0,1355.833333,1.0,0,9
9918,P53_FOV25_cell_7918,25,7918,C,1291.0,1284.888889,1.0,0,5
9919,P53_FOV25_cell_7977,25,7977,C,1349.0,10584.444444,1.0,0,0
9920,P53_FOV25_cell_8030,25,8030,C,1362.0,1515.111111,1.0,0,12


In [6]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters
0,1,3,A,1951.0,932.666667,1.0,0,16
1,1,10,A,1552.0,1307.444444,1.0,0,0
2,1,14,A,1636.0,1000.277778,1.0,0,7
3,1,44,A,2457.0,963.777778,1.0,0,11
4,1,63,A,2309.0,1052.222222,1.0,0,11
...,...,...,...,...,...,...,...,...
9917,25,7810,C,1130.0,1355.833333,1.0,0,9
9918,25,7918,C,1291.0,1284.888889,1.0,0,5
9919,25,7977,C,1349.0,10584.444444,1.0,0,0
9920,25,8030,C,1362.0,1515.111111,1.0,0,12


In [7]:
# add col to indicate if cell is predicted CD34+ lk cell
morph_predicted['Lk_cell'] = 0
morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,1,2,3,6,15])] = 1
morph_predicted

/tmp/ipykernel_1647515/2721977412.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,1,2,3,6,15])] = 1


,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
0,1,3,A,1951.0,932.666667,1.0,0,16,0
1,1,10,A,1552.0,1307.444444,1.0,0,0,1
2,1,14,A,1636.0,1000.277778,1.0,0,7,0
3,1,44,A,2457.0,963.777778,1.0,0,11,0
4,1,63,A,2309.0,1052.222222,1.0,0,11,0
...,...,...,...,...,...,...,...,...,...
9917,25,7810,C,1130.0,1355.833333,1.0,0,9,0
9918,25,7918,C,1291.0,1284.888889,1.0,0,5,0
9919,25,7977,C,1349.0,10584.444444,1.0,0,0,1
9920,25,8030,C,1362.0,1515.111111,1.0,0,12,0


In [8]:
morph_predicted[morph_predicted['Lk_cell'] == 1]

,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
1,1,10,A,1552.0,1307.444444,1.0,0,0,1
9,1,91,A,3038.0,1811.166667,1.0,0,1,1
20,1,252,A,2126.0,3611.333333,1.0,0,0,1
43,1,434,A,2076.0,2366.888889,1.0,0,1,1
46,1,443,A,2302.0,1551.722222,1.0,0,2,1
...,...,...,...,...,...,...,...,...,...
9911,25,7293,C,2379.0,1190.222222,1.0,0,2,1
9913,25,7334,C,655.0,2018.555556,1.0,0,2,1
9914,25,7423,C,752.0,2720.722222,1.0,0,15,1
9915,25,7512,C,2082.0,4569.055556,1.0,0,1,1


In [9]:
# save list of all patient's CD34+ cells
pos_to_save = morph_predicted[morph_predicted['Lk_cell'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely Lk cells

In [10]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 35 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of CD34+ cells
    pos_cells = combo[combo['Lk_cell'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('CD34_subclusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_CD34_subcluster.png', img)

20220215_094003_S2_C902_P99_N99_F001_normalized.png
20220215_094003_S2_C902_P99_N99_F001_normalized_cp_masks.tif


/tmp/ipykernel_1647515/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S2_C902_P99_N99_F002_normalized.png
20220215_094003_S2_C902_P99_N99_F002_normalized_cp_masks.tif


/tmp/ipykernel_1647515/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S2_C902_P99_N99_F003_normalized.png
20220215_094003_S2_C902_P99_N99_F003_normalized_cp_masks.tif


...

/tmp/ipykernel_1647515/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S2_C902_P99_N99_F024_normalized.png
20220215_094003_S2_C902_P99_N99_F024_normalized_cp_masks.tif


/tmp/ipykernel_1647515/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S2_C902_P99_N99_F025_normalized.png
20220215_094003_S2_C902_P99_N99_F025_normalized_cp_masks.tif


/tmp/ipykernel_1647515/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


In [11]:
# save list of CD34+ lk cells
pos_to_save.to_csv(patient + '_LkCells.csv', index=False)